Now, coming to Week 2 task. For each of the authors, you ideally want to know more about their background, topics of interest, twitter links etc.

One easy way to grab it is going their muckrack journalist profile page and web scrapping it from there (refer to chapter 2, beautifulsoup portion to know how to do it).

Two ways to do it: first, you can search through the muckrack sitemap CSV file for near duplicate matches in similar or same names and scrape their profiles. 

Second method is letting Bing/Google do the job aka take the author name and outlet name and search Google programmatically using Specrom APIs. For example if you want to know more about Dave Sebastian WSJ.com just use it as a search key and find muckrack profile and proceed from there. 

How about you trying doing do both approaches on top 10-30 authors and see which one works best?

https://algorithmia.com/algorithms/specrom/Google_scraper

https://algorithmia.com/algorithms/specrom/Bing_scraper

In [1]:
import pickle
import pandas as pd
import re
import Algorithmia
import bs4 as bs
import lxml
from urllib.request import Request, urlopen
import json
import time

In [2]:
with open('news.pkl', 'rb') as f:
    news = pickle.load(f)

muckrack = pd.read_csv('muckrack_persons_fetchlist.csv')

In [3]:
client = Algorithmia.client('simKLMHlWDMzWd+QcNRqVhI104r1')
algo = client.algo('specrom/Google_scraper/0.1.4')
algo.set_options(timeout=300) # optional

In [4]:
def get_top(n):
    temp = news[['author', 'article_count']].groupby('author').count().sort_values(['article_count'], ascending = False)
    temp = temp.reset_index().head(n)
    urls = []
    for i in range(len(temp)):
        url = re.sub(r'(\w*.\w*@\w*.\w*)', '', temp.loc[i].author.lower()) # remove emails
        url = re.sub(r'\|.*$', '', url) # remove everything after |
        url = re.sub(r'\,.*$', '', url) # remove evrything after ,
        url = re.sub(r'[^\w\s]', '', url) # remove punctuation
        url = ' '.join([i.strip() for i in url.split()]) # remove spaces and lowercase
        url = re.sub(r'(sa)(\s+)(\w+)(\s+)', '', url) # SA editors
        url = re.sub(r'(tulsa)(\s+)(world)', '', url) # tulsa world editors
        url = re.sub(r'(\d+)', '', url) # remove numbers
        url = 'https://muckrack.com/' + url.replace(' ', '-').strip('-') # make URL
        # print(muckrack[muckrack.url.str.contains(url, regex=False)])
        urls.append(url)
    temp['Muckrack'] = urls
    return temp

get_top(100)

,author,article_count,Muckrack
0,Neil Shaw,523,https://muckrack.com/neil-shaw
1,James Rodger,512,https://muckrack.com/james-rodger
2,Jack Davis,251,https://muckrack.com/jack-davis
3,Adam Wells,223,https://muckrack.com/adam-wells
4,Sophie McCoid,218,https://muckrack.com/sophie-mccoid
...,...,...,...
95,Joseph Chapman,112,https://muckrack.com/joseph-chapman
96,Valerie Edwards,112,https://muckrack.com/valerie-edwards
97,J.R. De Groote,112,https://muckrack.com/jr-de-groote
98,Clyde Hughes,111,https://muckrack.com/clyde-hughes


In [5]:
def extract_data(soup):
    info = {}
    info['name'] = data.iloc[j].author
    if soup.find_all(class_ = 'profile-verified'):
        info['verified'] = True
    if soup.find(class_ = 'person-details-item person-details-beats'):
        info['interests'] = ' '.join([line.strip() for line in soup.find(class_ = 'fa fa-fw fa-tasks icon-standard').findNext('div').text.split('\n') if line.strip() != ''])
    if soup.find(class_ = 'person-details-item person-details-title'):
        info['title'] = ' '.join([line.strip() for line in soup.find(class_ = 'fa fa-fw fa-building icon-standard').findNext('div').text.split('\n') if line.strip() != ''])
    if soup.find(class_ = 'person-details-item person-details-location'):
        info['location'] = ' '.join([line.strip() for line in soup.find(class_ = 'fa fa-fw fa-map-marker icon-standard').findNext('div').text.split('\n') if line.strip() != ''])
    if soup.find(class_ = 'mr-font-size-lg mr-font-family-2 top-sm'):
        info['description'] = soup.find(class_ = 'mr-font-size-lg mr-font-family-2 top-sm').text.strip()
    info['seen_in'] = list(set([i.text for i in soup.find_all('a') if '/media-outlet/' in i.attrs['href']]))
    if soup.find(class_ = 'profile-section-social'):
        for i in soup.find(class_ = 'profile-section-social'):
            temp = [j[5:] for j in str(i).split() if 'href' in j]
            if temp: 
                temp = temp[0].strip('"')
                if 't.co' in temp:
                    info['website'] = temp
                if 'twitter' in temp:
                    info['twitter'] = temp
                if 'linkedin' in temp:
                    info['linkedin'] = temp
                if 'facebook' in temp:
                    info['facebook'] = temp
                if 'instagram' in temp:
                    info['instagram'] = temp      
    print(info)

In [6]:
data = get_top(100)

In [7]:
for j in range(len(data)):
    try:
        url = data.iloc[j].Muckrack
        req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        sauce = urlopen(req).read()
    except:
        print("wrong url", data.iloc[j].Muckrack, data.iloc[j].author)
        query = {"query": f"{data.iloc[j].author} site: muckrack"}
        url = algo.pipe(query).result
        url = url[0]['url']
        if 'muckrack' in url:    
            print("right url", url)
            print()
            req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
            sauce = urlopen(req).read()
    soup = bs.BeautifulSoup(sauce,'lxml')
    extract_data(soup)
    print()

{'name': 'Neil Shaw', 'verified': True, 'interests': 'United Kingdom', 'title': 'Network Content Editor — DevonLive, Reach plc', 'location': 'England', 'description': 'Network Content Editor for Reach Plc. All of my words are second-hand, and useless in the face of this.', 'seen_in': ['Al Jazeera', 'The Washington Post', 'Reach plc', 'MSN South Africa', 'MSN UK', 'The (Toronto) Star', 'DevonLive', 'Fox News', 'Sunday Mirror', 'Associated Press', 'Daily Star (UK)', 'Daily Mirror'], 'website': 'https://t.co/dTqiiMyFdj', 'twitter': 'http://twitter.com/neiljshaw', 'linkedin': 'https://www.linkedin.com/in/neil-shaw-b45369a/'}

{'name': 'James Rodger', 'verified': True, 'interests': 'United Kingdom', 'title': 'Head of Trends — Birmingham Live', 'location': 'Birmingham', 'description': 'Regional Content Editor for @birmingham_live.', 'seen_in': ['Western Mail', 'Rochdale Observer', 'MSN South Africa', 'Birmingham Live', 'MSN UK', 'Liverpool Echo', 'Sunday Mercury', 'Sunday Mirror', 'Daily Sta

{'name': 'Lewis Winter', 'verified': True, 'interests': 'Arts and Entertainment, Sports, United Kingdom', 'title': 'Sports Reporter — Daily Express', 'location': 'Woking', 'description': 'Digital sports reporter @DExpress_Sport. Arsenal ST holder, Woking fan and Surrey CCC member. Views mine.', 'seen_in': ['Daily Express', 'Daily Star (UK)', 'Bleacher Report'], 'website': 'https://t.co/pXLZln5ewh', 'twitter': 'http://twitter.com/LewisAWinter', 'linkedin': 'https://www.linkedin.com/in/lewis-winter-b0081b130/?originalSubdomain=uk'}

{'name': 'Rob Quinn', 'verified': True, 'interests': 'Arts and Entertainment, China', 'title': 'Reporter and Editor — Newser', 'location': 'Hong Kong', 'seen_in': ['KGW-TV (Portland, OR)', 'Detroit Free Press', 'Times Higher Education (THE)', 'The Globe and Mail', 'USA Today', 'Newser', 'Yahoo', 'WXIA-TV (Atlanta, GA)', 'Fox News', 'KHOU-TV (Houston, TX)', 'The Huffington Post'], 'linkedin': 'https://hk.linkedin.com/in/rob-quinn-57732a12'}

{'name': 'Tom Bull

wrong url https://muckrack.com/matthew-cooper Matthew Cooper
right url https://muckrack.com/matt-cooper-2

{'name': 'Matthew Cooper', 'verified': True, 'interests': 'Arts and Entertainment, Metro Los Angeles', 'title': 'Listings Coordinator — Los Angeles Times', 'location': 'Los Angeles', 'seen_in': ['Chicago Tribune', 'The San Diego Union-Tribune', 'Los Angeles Times', 'Orlando Sentinel', 'MSN', 'Yahoo', 'Yahoo Canada'], 'linkedin': 'https://www.linkedin.com/in/matthew-cooper-13860015/'}

{'name': 'Helen Kelly', 'verified': True, 'interests': 'United Kingdom', 'title': 'TV Editor — Daily Express', 'location': 'United Kingdom', 'description': 'TV Editor @Daily_Express 📺 All views are my own', 'seen_in': ['Daily Express', 'LancsLive', 'Lexology', 'Liverpool Echo', 'Informa', 'Scoop', 'Daily Star (UK)'], 'website': 'http://alittlepieceofhelen.blogspot.co.uk/', 'twitter': 'http://twitter.com/helenm_kelly'}

{'name': 'Chris Smith', 'location': 'San Ramon', 'description': 'Head of SEO @ Hou

{'name': 'Ollie Salt', 'verified': True, 'interests': 'Sports', 'title': 'Sport Reporter — Daily Star (UK)', 'location': 'London', 'description': '@DailyStar_Sport reporter covering football/boxing. ollie.salt@reachplc.com', 'seen_in': ['Boxing Scene', 'Daily Star (UK)'], 'twitter': 'http://twitter.com/olliesalt', 'linkedin': 'https://www.linkedin.com/in/ollie-salt-a5a0b8111'}

{'name': "Sean O'grady", 'verified': True, 'interests': 'Business and Finance, United Kingdom', 'title': 'Associate Editor — Irish Independent', 'location': 'London', 'description': 'Associate Editor at the @independent, formerly Economics Editor. Views my own.', 'seen_in': ['The Independent', 'Daily Mail', 'Yahoo Singapore', 'MSN South Africa', 'MSN UK', 'MSN Australia', 'MSN India', 'MSN', 'Yahoo Canada', 'Yahoo News UK', 'Irish Independent'], 'twitter': 'http://twitter.com/_SeanOGrady', 'linkedin': 'https://www.linkedin.com/in/sean-o-grady-61512030'}

{'name': 'Claire Anderson', 'seen_in': ['Rochdale Observer

{'name': 'Dan Bloom', 'verified': True, 'interests': 'Politics, United Kingdom', 'title': 'Online Political Editor — Daily Mirror', 'description': 'Online political editor, Daily Mirror. I rise early to feed the beast of @mirrorpolitics. Story tips to dan.bloom@mirror.co.uk; DMs open', 'seen_in': ['Daily Mail', 'MSN South Africa', 'MSN UK', 'Daily Star (UK)', 'Birmingham Live', 'Reuters', 'MSN', 'Sunday Mirror', 'Medium', 'Wales on Sunday', 'Daily Mirror'], 'twitter': 'http://twitter.com/danbloom1', 'facebook': 'https://www.facebook.com/mirrordanbloom/', 'linkedin': 'https://uk.linkedin.com/in/dan-bloom-7a0192134'}

{'name': 'Brian Mazique', 'verified': True, 'interests': 'Sports, U.S.', 'title': 'Freelance Sports and Video Games Journalist — Freelance', 'location': 'Chicago', 'description': 'Sports & Gaming Journalist @Forbes @HeavySan #NBA2K #Madden #Bulls #Bears #Patriots - Here by the grace of God', 'seen_in': ['Heavy.com', 'Bleacher Report', 'Freelance', 'Hot Virals', 'Forbes', 'T

{'name': 'Shane Ireland', 'verified': True, 'interests': 'United Kingdom', 'title': 'Journalist and Write — Birmingham Live', 'location': 'Warwick', 'description': 'Journalist @Birmingham_Live covering football clubs across the Midlands.', 'seen_in': ['The Sun', 'Coventry Telegraph', 'Birmingham Live', 'MSN UK', 'Bristol Post', 'Derby Telegraph', 'Cambridge News', 'Sunday Mercury', 'Nottingham Post', 'Mercury Extra', 'Daily Mirror'], 'website': 'https://t.co/G4KoFhgh9j', 'twitter': 'http://twitter.com/_ShaneIreland', 'facebook': 'http://facebook.com/ShaneIrelandJournalist', 'linkedin': 'https://www.linkedin.com/in/shane-ireland-b26a39100'}

{'name': 'Evann Gastaldo', 'verified': True, 'interests': 'U.S.', 'title': 'Managing Editor — Newser', 'location': 'Philadelphia', 'description': 'Managing Editor @Newser. As the main gossip writer, I know way more about Justin Bieber than I ever wanted to.', 'seen_in': ['Arizona Republic', 'AZCentral.com', 'WUSA-TV (Washington, DC)', 'WKYC-TV (Clev